In [32]:
import ee
import numpy as np
import pandas as pd
import datetime
import geemap
import json
import folium
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [15]:
ee.Authenticate()

True

In [16]:
# Initialize Earth Engine
ee.Initialize()

In [ ]:
# Define the Corn Belt states
states = ["Iowa"]

# Define the date range for the full month of August 2022
dates = pd.date_range(start="2022-06-01", end="2022-08-31", freq='W')

# Helper function to find the closest available Sentinel-2 image
def get_closest_sentinel_image(county_geom, date):
    """Find the closest available Sentinel-2 image."""
    start_date = (pd.to_datetime(date) - pd.Timedelta(days=3)).strftime('%Y-%m-%d')
    end_date = (pd.to_datetime(date) + pd.Timedelta(days=3)).strftime('%Y-%m-%d')

    sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(county_geom) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .sort('system:time_start') \
        .first()

    if sentinel.getInfo() is None:
        print(f"No Sentinel-2 image available between {start_date} and {end_date}.")
        return None

    band_names = sentinel.bandNames().getInfo()
    required_bands = {'B2', 'B3', 'B4', 'B8', 'B11'}
    if not required_bands.issubset(band_names):
        print(f"Sentinel-2 image is missing one or more required bands.")
        return None

    return sentinel

# Helper function to calculate relevant satellite indices
def calculate_indices(sentinel):
    """Calculate NDVI, EVI, GNDVI, NDWI, and SAVI from Sentinel-2 data."""
    ndvi = sentinel.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = sentinel.expression(
        '2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))',
        {'B8': sentinel.select('B8'), 'B4': sentinel.select('B4'), 'B2': sentinel.select('B2')}
    ).rename('EVI')
    gndvi = sentinel.normalizedDifference(['B8', 'B3']).rename('GNDVI')
    ndwi = sentinel.normalizedDifference(['B8', 'B11']).rename('NDWI')
    savi = sentinel.expression(
        '((B8 - B4) / (B8 + B4 + 0.5)) * 1.5',
        {'B8': sentinel.select('B8'), 'B4': sentinel.select('B4')}
    ).rename('SAVI')

    return ndvi.addBands([evi, gndvi, ndwi, savi])



In [50]:
# Load the USDA CSBAL1623 field boundaries dataset
FIELD_BOUNDARIES_ASSET = 'projects/nass-csb/assets/csb1623/CSBIA1623'
field_boundaries = ee.FeatureCollection(FIELD_BOUNDARIES_ASSET)

# 1. Explore all features in the FeatureCollection
def explore_features(feature_collection):
    """Print and explore features and their properties."""
    first_feature = feature_collection.first()
    properties = first_feature.getInfo()['properties']

# Call the function to explore features
explore_features(field_boundaries)

# 2. Select a specific feature (for example, the first one)
selected_feature = field_boundaries.first()

# 3. Convert the selected feature's geometry to GeoJSON
feature_geojson = selected_feature.geometry().getInfo()

# 4. Visualize the feature with Folium
def visualize_feature_with_folium(geojson):
    """Visualize a feature on a Folium map."""
    # Create a Folium map centered on the feature's coordinates
    center = geojson['coordinates'][0][0]  # Extract the coordinates of the feature
    m = folium.Map(location=[center[1], center[0]], zoom_start=12)

    # Add the GeoJSON feature to the map
    folium.GeoJson(geojson, name="Selected Feature").add_to(m)

    # Display the map
    return m

# Display the selected feature on a Folium map
visualize_feature_with_folium(feature_geojson)

Properties of the first feature:
{
  "ACRES": 11.72,
  "ASD": "10",
  "CDL2016": 1,
  "CDL2017": 5,
  "CDL2018": 1,
  "CDL2019": 5,
  "CDL2020": 5,
  "CDL2021": 36,
  "CDL2022": 36,
  "CDL2023": 36,
  "CNTY": "Dickinson",
  "CNTYFIPS": "059",
  "CROP16": "Corn",
  "CROP17": "Soybeans",
  "CROP18": "Corn",
  "CROP19": "Soybeans",
  "CROP20": "Soybeans",
  "CROP21": "Alfalfa",
  "CROP22": "Alfalfa",
  "CROP23": "Alfalfa",
  "CSBACRES": 11.7187114631,
  "CSBID": "191623001929176",
  "CSBYEARS": "1623",
  "INSIDE_X": 65952.2226321,
  "INSIDE_Y": 2268095.76203,
  "STATEASD": "1910",
  "STATEFIPS": "19",
  "Shape_Area": 47423.9846716,
  "Shape_Leng": 942.25795455,
  "endyear": 2023,
  "fid": 202872,
  "startyear": 2016
}


In [ ]:


# 2. Select a specific feature (for example, the first one)
selected_feature = field_boundaries.first()

# 3. Convert the selected feature's geometry to GeoJSON
feature_geojson = selected_feature.geometry().getInfo()

# 4. Visualize the feature with Folium
def visualize_feature_with_folium(geojson):
    """Visualize a feature on a Folium map."""
    # Create a Folium map centered on the feature's coordinates
    center = geojson['coordinates'][0][0]  # Extract the coordinates of the feature
    m = folium.Map(location=[center[1], center[0]], zoom_start=12)

    # Add the GeoJSON feature to the map
    folium.GeoJson(geojson, name="Selected Feature").add_to(m)

    # Display the map
    return m

# Display the selected feature on a Folium map
visualize_feature_with_folium(feature_geojson)